In [204]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = (
    SparkSession.builder
    .master('spark://172.19.0.4:7777')
    .appName('ETL_LAB03')
    .getOrCreate()
)

In [205]:
bank_df = spark.read.csv('../data/bronze/banks/EnquadramentoInicia_v2.tsv', header=True, sep='\t')

# Data Transformation for bank dataset
for column in bank_df.columns:
    bank_df = bank_df.withColumnRenamed(
        existing=column, 
        new=column.lower()
    )

for replacement_action in [
    ("nome", "- PRUDENCIAL", ""),
    ("nome","(\.+|\/+|\-+)", ""),
]:
    bank_df = bank_df.withColumn(
        "nome", regexp_replace(
            replacement_action[0],
            replacement_action[1],
            replacement_action[2]
        )
    )

print(bank_df.count())
# bank_df = bank_df.na.drop()
# bank_df = bank_df.dropDuplicates()
bank_df.show(5, truncate=False)
print(f"Number of rows x columns - Bank Data: {bank_df.count()} x {len(bank_df.columns)}")
bank_df.write.mode("overwrite").csv("../data/silver/banks", header=True, sep=";")

1474
+--------+--------+------------------------+
|segmento|cnpj    |nome                    |
+--------+--------+------------------------+
|S1      |0       |BANCO DO BRASIL         |
|S1      |60746948|BRADESCO                |
|S1      |30306294|BTG PACTUAL             |
|S1      |360305  |CAIXA ECONOMICA FEDERAL |
|S1      |60872504|ITAU                    |
+--------+--------+------------------------+
only showing top 5 rows

Number of rows x columns - Bank Data: 1474 x 3


In [206]:
# Lendo separado devido a estrutura diferente
employee_df_1 = spark.read.format("csv").option("header", "true").option('delimiter','|').load("../data/bronze/employees/glassdoor_consolidado_join_match_less_v2.csv")
employee_df_2 = spark.read.format("csv").option("header", "true").option('delimiter','|').load("../data/bronze/employees/glassdoor_consolidado_join_match_v2.csv")

# Criando colunas
employee_df_1 = employee_df_1.withColumn('Segmento', lit(''))
employee_df_2 = employee_df_2.withColumn('CNPJ', lit(''))

# Ordenando as colunas
columns = ["employer_name", "reviews_count", "culture_count", "salaries_count", "benefits_count", "employer-website", "employer-headquarters", "employer-founded", "employer-industry", "employer-revenue", "url", "Geral", "Cultura e valores", "Diversidade e inclusão", "Qualidade de vida", "Alta liderança", "Remuneração e benefícios", "Oportunidades de carreira", "Recomendam para outras pessoas(%)", "Perspectiva positiva da empresa(%)", "CNPJ", "Segmento", "Nome", "match_percent"]

employee_df_1 = employee_df_1.select(columns)
employee_df_2 = employee_df_2.select(columns)

# unindo os dados
employee_df = employee_df_1.union(employee_df_2)

# Data Transformation for employee dataset
for column in employee_df.columns:
    employee_df = employee_df.withColumnRenamed(
        column, 
        column.replace("-","_").replace(" ","_").lower()
    )

employee_df.show(truncate=False)
employee_df.cache()

for replacement_action in [
    ("nome", "- PRUDENCIAL", ""),
    ("nome","(\.+|\/+|\-+)", ""),
]:
    employee_df = employee_df.withColumn(
        "nome", regexp_replace(
            replacement_action[0],
            replacement_action[1],
            replacement_action[2]
        )
    )

print(employee_df.count())
# employee_df = employee_df.na.drop()
# employee_df = employee_df.dropDuplicates()
employee_df.show(5, truncate=False)
print(f"Number of rows x columns - Employee Data: {employee_df.count()} x {len(employee_df.columns)}")
employee_df.write.mode("overwrite").csv("../data/silver/employee", header=True, sep=";")

+-------------------------------+-------------+-------------+--------------+--------------+---------------------------------------------------+---------------------+----------------+-----------------------------------------------------------------------------------+---------------------------+----------------------------------------------------------------------------------------------------------------+-----+-----------------+----------------------+-----------------+--------------+------------------------+-------------------------+---------------------------------+----------------------------------+--------+--------+--------------------------------------------------+-------------+
|employer_name                  |reviews_count|culture_count|salaries_count|benefits_count|employer_website                                   |employer_headquarters|employer_founded|employer_industry                                                                  |employer_revenue           |url            

In [208]:
#Claims
claims_df = spark.read.format("csv").option("header", "true").option('delimiter',',').load("../data/bronze/claims")

#print(claims_df.count()) == 918

# Data Transformation for employee dataset
for column in claims_df.columns:
    claims_df = claims_df.withColumnRenamed(
        column, 
        column.replace("-","_").replace(" ","_").lower()
    )

claims_df = claims_df.withColumnRenamed('cnpj_if', 'cnpj')\
    .withColumnRenamed('instituição_financeira', 'nome')

for replacement_action in [
    ("nome", "- PRUDENCIAL", ""),
    ("nome","(\.+|\/+|\-+)", ""),
    ("nome"," \(conglomerado\)", ""),
]:
    claims_df = claims_df.withColumn(
        "nome", regexp_replace(
            replacement_action[0],
            replacement_action[1],
            replacement_action[2]
        )
    )

# claims_df = claims_df.na.drop()
# claims_df = claims_df.dropDuplicates()
claims_df.show(5, truncate=False)
print(f"Number of rows x columns - Employee Data: {employee_df.count()} x {len(employee_df.columns)}")
claims_df.write.mode("overwrite").csv("../data/silver/claims", header=True, sep=";")

+----+---------+------------------------------------------------------+----------------+--------+---------------------------------------------------------------+------+-----------------------------------------------+--------------------------------------------+---------------------------------------+-------------------------------+----------------------------------------+----------------------------+----------------------------+
|ano |trimestre|categoria                                             |tipo            |cnpj    |nome                                                           |índice|quantidade_de_reclamações_reguladas_procedentes|quantidade_de_reclamações_reguladas___outras|quantidade_de_reclamações_não_reguladas|quantidade_total_de_reclamações|quantidade_total_de_clientes_–_ccs_e_scr|quantidade_de_clientes_–_ccs|quantidade_de_clientes_–_scr|
+----+---------+------------------------------------------------------+----------------+--------+-------------------------------------

In [209]:
# reading data for joining
claims_df = spark.read.format("csv").option("header", "true").option('delimiter',';').load("../data/silver/claims")
employee_df = spark.read.format("csv").option("header", "true").option('delimiter',';').load("../data/silver/employee")
employee_df = employee_df.drop('cnpj','segmento')
banks_df = spark.read.format("csv").option("header", "true").option('delimiter',';').load("../data/silver/banks")


In [210]:
claims_df.show(100, truncate=False)

+----+---------+--------------------------------------------------------+----------------+--------+---------------------------------------------------------------+------+-----------------------------------------------+--------------------------------------------+---------------------------------------+-------------------------------+----------------------------------------+----------------------------+----------------------------+
|ano |trimestre|categoria                                               |tipo            |cnpj    |nome                                                           |índice|quantidade_de_reclamações_reguladas_procedentes|quantidade_de_reclamações_reguladas___outras|quantidade_de_reclamações_não_reguladas|quantidade_total_de_reclamações|quantidade_total_de_clientes_–_ccs_e_scr|quantidade_de_clientes_–_ccs|quantidade_de_clientes_–_scr|
+----+---------+--------------------------------------------------------+----------------+--------+-------------------------------

In [211]:
claims_df.printSchema()

root
 |-- ano: string (nullable = true)
 |-- trimestre: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- cnpj: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- índice: string (nullable = true)
 |-- quantidade_de_reclamações_reguladas_procedentes: string (nullable = true)
 |-- quantidade_de_reclamações_reguladas___outras: string (nullable = true)
 |-- quantidade_de_reclamações_não_reguladas: string (nullable = true)
 |-- quantidade_total_de_reclamações: string (nullable = true)
 |-- quantidade_total_de_clientes_–_ccs_e_scr: string (nullable = true)
 |-- quantidade_de_clientes_–_ccs: string (nullable = true)
 |-- quantidade_de_clientes_–_scr: string (nullable = true)


In [212]:
employee_df.show(truncate=False)

+----------------------------+-------------+-------------+--------------+--------------+-------------------------------+---------------------------+----------------+-----------------------------------------------------------------------------------+---------------------------+-------------------------------------------------------------------------------------------------------------+-----+-----------------+----------------------+-----------------+--------------+------------------------+-------------------------+---------------------------------+----------------------------------+------------------------------+-------------+
|employer_name               |reviews_count|culture_count|salaries_count|benefits_count|employer_website               |employer_headquarters      |employer_founded|employer_industry                                                                  |employer_revenue           |url                                                                                       

In [213]:
employee_df.printSchema()

root
 |-- employer_name: string (nullable = true)
 |-- reviews_count: string (nullable = true)
 |-- culture_count: string (nullable = true)
 |-- salaries_count: string (nullable = true)
 |-- benefits_count: string (nullable = true)
 |-- employer_website: string (nullable = true)
 |-- employer_headquarters: string (nullable = true)
 |-- employer_founded: string (nullable = true)
 |-- employer_industry: string (nullable = true)
 |-- employer_revenue: string (nullable = true)
 |-- url: string (nullable = true)
 |-- geral: string (nullable = true)
 |-- cultura_e_valores: string (nullable = true)
 |-- diversidade_e_inclusão: string (nullable = true)
 |-- qualidade_de_vida: string (nullable = true)
 |-- alta_liderança: string (nullable = true)
 |-- remuneração_e_benefícios: string (nullable = true)
 |-- oportunidades_de_carreira: string (nullable = true)
 |-- recomendam_para_outras_pessoas(%): string (nullable = true)
 |-- perspectiva_positiva_da_empresa(%): string (nullable = true)
 |-- nom

In [214]:
banks_df.show(truncate=False)

+--------+--------+-------------------------------+
|segmento|cnpj    |nome                           |
+--------+--------+-------------------------------+
|S1      |0       |BANCO DO BRASIL                |
|S1      |60746948|BRADESCO                       |
|S1      |30306294|BTG PACTUAL                    |
|S1      |360305  |CAIXA ECONOMICA FEDERAL        |
|S1      |60872504|ITAU                           |
|S1      |90400888|SANTANDER                      |
|S2      |92702067|BANRISUL                       |
|S2      |7237373 |BANCO DO NORDESTE DO BRASIL SA |
|S2      |33657248|BNDES                          |
|S2      |33479023|CITIBANK                       |
|S2      |33987793|CREDIT SUISSE                  |
|S2      |58160789|SAFRA                          |
|S2      |59588111|VOTORANTIM                     |
|S3      |28195667|ABCBRASIL                      |
|S3      |60770336|ALFA                           |
|S3      |655522  |APE POUPEX                     |
|S3      |29

In [215]:
banks_df.printSchema()

root
 |-- segmento: string (nullable = true)
 |-- cnpj: string (nullable = true)
 |-- nome: string (nullable = true)


In [216]:
join_df = claims_df.join(banks_df, ['nome','cnpj'], 'inner' )

In [217]:
join_df.show(50, truncate=False)

+-------------------------------+--------+----+---------+------------------------------------------------------+----------------+------+-----------------------------------------------+--------------------------------------------+---------------------------------------+-------------------------------+----------------------------------------+----------------------------+----------------------------+--------+
|nome                           |cnpj    |ano |trimestre|categoria                                             |tipo            |índice|quantidade_de_reclamações_reguladas_procedentes|quantidade_de_reclamações_reguladas___outras|quantidade_de_reclamações_não_reguladas|quantidade_total_de_reclamações|quantidade_total_de_clientes_–_ccs_e_scr|quantidade_de_clientes_–_ccs|quantidade_de_clientes_–_scr|segmento|
+-------------------------------+--------+----+---------+------------------------------------------------------+----------------+------+--------------------------------------------

In [268]:
final_df = join_df.join( employee_df, 'nome' , 'inner')

In [257]:
final_df.show(truncate=False)

+------------------------------+--------+----+---------+------------------------------------------------------+----------------+------+-----------------------------------------------+--------------------------------------------+---------------------------------------+-------------------------------+----------------------------------------+----------------------------+----------------------------+--------+---------------------------+-------------+-------------+--------------+--------------+----------------------------+---------------------+----------------+-----------------------------------------------------------------------------------+--------------------------+------------------------------------------------------------------------------------------------------------+-----+-----------------+----------------------+-----------------+--------------+------------------------+-------------------------+---------------------------------+----------------------------------+-------------+
|no

In [269]:
# Estrutura final esperada:
# Nome do Banco
# CNPJ
# Classificação do Banco
# Quantidade de Clientes do Bancos
# Índice de reclamações
# Quantidade de reclamações
# Índice de satisfação dos funcionários dos bancos
# Índice de satisfação com salários dos funcionários dos bancos.

final_df = final_df.select(col('nome').alias('Nome do Banco'), 
                           col('cnpj').alias('CNPJ'),
                           col('categoria').alias('Classificação'),
                           col('quantidade_total_de_clientes_–_ccs_e_scr').alias('Quantidade de Clientes do Bancos'),
                           col('índice').alias('Índice de reclamações'),
                           col('quantidade_total_de_reclamações').alias('Quantidade de reclamações'),
                           col('geral').alias('Índice de satisfação dos funcionários dos bancos'),
                           col('remuneração_e_benefícios').alias('Índice de satisfação com salários dos funcionários dos bancos'))

final_df = final_df.groupBy('Nome do Banco',
                            'CNPJ',
                            'Classificação')\
                   .agg(round(avg('Quantidade de Clientes do Bancos')).alias('Quantidade de Clientes do Bancos'),
                              avg('Índice de reclamações').alias('Índice de reclamações'),
                              avg('Quantidade de reclamações').alias('Quantidade de reclamações'),
                              avg('Índice de satisfação dos funcionários dos bancos').alias('Índice de satisfação dos funcionários dos bancos'),
                              avg('Índice de satisfação com salários dos funcionários dos bancos').alias('Índice de satisfação com salários dos funcionários dos bancos'))

+------------------------------+--------+------------------------------------------------------+--------------------------------+---------------------+-------------------------+------------------------------------------------+-------------------------------------------------------------+
|Nome do Banco                 |CNPJ    |Classificação                                         |Quantidade de Clientes do Bancos|Índice de reclamações|Quantidade de reclamações|Índice de satisfação dos funcionários dos bancos|Índice de satisfação com salários dos funcionários dos bancos|
+------------------------------+--------+------------------------------------------------------+--------------------------------+---------------------+-------------------------+------------------------------------------------+-------------------------------------------------------------+
|BANCO FIBRA SA                |58616418|Grupo Secundário                                      |163619.0                        |null

In [270]:
final_df.write.mode("overwrite").csv("../data/gold/final_table", header=True, sep=";")